PYTANIE: Czego potrzebujemy?
ODPOWIEDZ: Uczenie nienadzorowanego!

PYTANIE: Czemu?
ODPOWIEDZ: ...

PYTANIE: Czego użyjemy?
ODPOWIEDZ: Używamy GAN.

PYTANIE: Czemu GAN?
ODPOWIEDZ: Ponieważ ogólna zasada jest taka, że próbujemy coś wygenerować i mamy detykowanego agenta, który mówi czy to się udało czy nie.
Brzmi ok do uczenia generowania obrazów już istniejących.

PYTANIE: W takim razie jak to działa że tworzy nowe obrazy?
ODPOWIEDZ: Mamy w sumie dwa modele, jeden ma generować np. kwiatki, a drugi ma mówić czy to co dostaje do kwiatek. Jak model rozpoznający kwiatki powie "to nie kwiatek" to model generujący się updatuje, a jak model rozpoznający powie "to totalnie jest kwiatek" na to co dał model generujący, to model rozpoznający się updatuje i tak w kółko...
Na sam koniec mamy model, który dostając obraz na wejście "przewiduje" co może być na wyjściu, czyli na nasze, będzie tworzył obraz jaki "być powinien" na bazie tego czego się nauczył.

PYTANIE: A jakie sieci w tym GAN pan ma?
ODPOWIEDZ: convolutional neural network (CNN)

PYTANIE: Czemu?
ODPOWIEDŹ: CNN mogą uczyć się na "historii" czyli trochę tak jakby to był film o kwiatkach i model będzie przewidywał następną klatkę tego filmu o kwiatkach. Żeby to zrobić musi coś wygenerować i nam zaproponować. To tutaj mamy część "twórczą".
